In [16]:
from mpmath import zeta
import numpy as np 
import sympy
import matplotlib.pyplot as plt 
import pyprimes
import seaborn as sns

In [17]:
# Developed Libraries
from Zeta_Algorithms import zetaStochastic, zetaNormRange

In [9]:
# <img src="zeta_critical_line.jpg" width="600" height="600">

**The Riemann Zeta function is defined as:**
$$\zeta(s):=\sum_{n=1}^\infty{n^{-s}} \ \ \ \ : \ (s=\sigma + it) \ \ ; \ (\sigma > 1)$$

Our Zeta Stochastic Function is defined $(\ X_T(h),\  h\in I)$, this follows as 
$$X_T(h) = \sum_{p\leq T}\cdot \frac{1}{\sqrt{p}}Re(p^{ih}e^{i\theta_p})$$
Where $p$ refers to the primes, $Re$ is concerned witht the 'real' part of the expression $p^{ih}e^{i\theta_p}$ and $\theta_p \sim U(0,2\pi) \ IID$

Through... we have the following
$$X_T(h) = \sum_{p\leq T}\cdot \frac{1}{\sqrt{p}}cos(\theta_p - hlog(p))$$

In [10]:
# Stores prime list up to 10^8
# This minimizes runtime by limiting the creation of the prime list 
prime_num = np.array(list(pyprimes.primes_below(1e8)))

In [11]:
prime_num

array([       2,        3,        5, ..., 99999959, 99999971, 99999989])

In [ ]:
T_val = int(1e6)

# create range interval of h from -1 to 1
x=np.arange(-1,1,1/100)

# create stochastic zeta range over h interval  
sims = 10
y = np.array([[zetaStochastic(h=i, T=T_val, prime_list=prime_num) for i in x] for _ in range(sims)]).reshape(sims, 200)

In [ ]:
y

In [ ]:
# rolling maximum of norm function
maximum = [y[:, :i+1].max() for i in range(y.shape[1])]

# rolling minimum of norm function
minimum = [y[:, :i+1].min() for i in range(y.shape[1])]

In [ ]:
plt.figure(figsize=(20,10))  
for i in y:
    plt.plot(x, i)
plt.plot(x, np.average(y, axis=0), lw=3, color='black', label='Rolling Average')
plt.plot(x, maximum, lw=2, color='red', ls='--', label='Rolling Maximum')
plt.plot(x, minimum, lw=2, color='red', ls='--', label='Rolling Minimum')


plt.ylabel('Log Zeta Stochastic Value')
plt.xlabel('$h$ value')
plt.legend()
plt.show()

**Computations of the Riemann Zeta function on the critical line refer to computations of the form $\zeta(\frac{1}{2} + it)$, where we move vertically along the complex plane with $t$, while being fixed at the point $\frac{1}{2}$ on the real number line. We will be using an existing implementation of the Zeta function in Python, defined in the documentation provided, http://mpmath.org/doc/current/functions/zeta.html?highlight=zeta#mpmath.zeta** 



In [18]:
N = 0   # starting value of T range 
deltaN = 500    # length of T range

In [19]:
# creating the interval range t
t_range=np.arange(N, N+deltaN)

In [20]:
# storage for the normal zeta 
norm_zeta_value = zetaNormRange(N, deltaN)

# rolling average of norm function 
average = np.cumsum(norm_zeta_value)/(t_range+1)

# rolling maximum of norm function
maximum = [norm_zeta_value[:i+1].max() for i in range(norm_zeta_value.size)]

AttributeError: module 'numpy' has no attribute 'zeta'

In [15]:
plt.figure(figsize=(20,10))  
plt.plot(t_range, norm_zeta_value, label='Normalized Zeta')
plt.plot(t_range, average, label='Normalized Average', lw=3, color='black')
plt.plot(t_range, maximum, label='Normalized Maximum', lw=2, color='green')
plt.xlabel('T range')
plt.legend()
plt.show()

NameError: name 'norm_zeta_value' is not defined

<Figure size 1440x720 with 0 Axes>

In [ ]:
plt.figure(figsize=(20,8))
# deprecation error is an issue with displot implementation within Seaborn
sns.distplot(norm_zeta_value, bins=100, color='red')
plt.show()